In [ ]:
## imports
import os

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import jax
import jax.numpy as jnp
from jax.ops import index, index_update
import numpy as np
import pandas as pd
from scipy.io import loadmat

from pycbem.field import efield, hfield
from pycbem.utils.dataloader import load_antenna_el_properties
from pycbem.utils.viz import fig_config

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
coord_dict = loadmat(os.path.join('data', 'sphere', 'sphere_coord_n1512.mat'))
r_c = pd.DataFrame(coord_dict['r_c'] / 100., columns=['x', 'y', 'z'])

In [ ]:
f = 3.5e9
d = - 10 / 1000
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j
xs = antenna_data.x.to_numpy()
xs = xs - xs.max() / 2
ys = np.zeros_like(xs) + r_c['y'].min() + d
zs = np.zeros_like(xs)

In [ ]:
fig_config(latex=True, scaler=2.5)
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.plot(r_c['x'], r_c['y'], r_c['z'], 'b.')
ax.plot(xs, ys, zs, 'r-', linewidth=3)
ax.set(xlabel='x', ylabel='y', zlabel='z')
plt.show()

In [ ]:
xs = jnp.asarray(xs)
ys = jnp.asarray(ys)
zs = jnp.asarray(zs)
Is = jnp.asarray(Is)

In [ ]:
E = r_c.apply(
    lambda row: efield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
E.columns = ['Ex', 'Ey', 'Ez']
E_abs = E.apply(
    lambda row: np.sqrt(row['Ex'] ** 2 + row['Ey'] ** 2 + row['Ez'] ** 2),
    axis=1)
E.loc[:, 'E_abs'] = E_abs

H = r_c.apply(
    lambda row: hfield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
H.columns = ['Hx', 'Hy', 'Hz']
H_abs = H.apply(
    lambda row: np.sqrt(row['Hx'] ** 2 + row['Hy'] ** 2 + row['Hz'] ** 2),
    axis=1)
H.loc[:, 'H_abs'] = H_abs

r_c_calc = pd.concat([r_c, E, H], axis=1)

In [ ]:
def power_density(row):
    return (row.Ey * row.Hz.conjugate() - row.Ez * row.Hy.conjugate(),
            row.Ex * row.Hz.conjugate(),
            row.Ex * row.Hy.conjugate())


S = r_c_calc.apply(power_density, axis=1, result_type='expand')
S.columns = ['Sx', 'Sy', 'Sz']
S_abs = S.apply(
    lambda row: np.sqrt(row['Sx'] ** 2 + row['Sy'] ** 2 + row['Sz'] ** 2),
    axis=1)
S.loc[:, 'S_abs'] = S_abs 

r_c_calc = pd.concat([r_c_calc, S], axis=1)

In [ ]:
fig_config(latex=True, scaler=2.5)
fig = plt.figure()
ax = plt.axes(projection ='3d')

cs = ax.scatter(r_c_calc['x'], r_c_calc['y'], r_c_calc['z'],
                c=np.abs(r_c_calc['S_abs']))
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel('$S$ [W/m$^2$]')

ax.plot(xs, ys, zs, linestyle='-', c='r', zorder=3,
        label=(f'$\\lambda/2$ dipole antenna\n'
               f'$d = {-d * 1000}$ mm\n'
               f'$f = {f / 1e9}$ GHz'))

ax.set(xlabel='$x$', ylabel='$y$', zlabel='z')
plt.legend()
plt.show()

In [ ]:
area = (0.02, 0.02)
A = area[0] * area[1]
xy_area = (-area[0]/2, -area[1]/2)

fig_config(latex=True, scaler=2)
fig = plt.figure()
ax = fig.add_subplot()
cs = ax.scatter(r_c_calc['x'], r_c_calc['z'], c=np.abs(r_c_calc['S_abs']))
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel('$S$ [W/m$^2$]')
ax.plot(xs, zs, linestyle='-', c='r',
        label=(f'$\\lambda/2$ dipole antenna\n'
               f'$d = {-d * 1000}$ mm\n'
               f'$f = {f / 1e9}$ GHz'))
ax.add_patch(Rectangle(xy_area, area[0], area[1], ec='w', fc='None',
                       label=f'effective area\n$A = {A * 1e4}$ cm$^2$'))
ax.set(xlabel='$x$', ylabel='z')
plt.legend(facecolor='darkslateblue', labelcolor='white', framealpha=0.9)
plt.show()

In [ ]:
r_c_calc_ta = r_c_calc[
    (r_c_calc['y'] < 0) &
    (r_c_calc['x'] > xy_area[0]) & (r_c_calc['x'] < xy_area[0] * -1) &
    (r_c_calc['z'] > xy_area[1]) & (r_c_calc['z'] < xy_area[1] * -1)]
r_c_calc_ta.reset_index(drop=True, inplace=True)

In [ ]:
r_c_calc_ta